# 데이터 파악

In [131]:
import pandas as pd
channel = pd.read_csv('dataset\채널이용.csv', encoding='utf-8')
compuse = pd.read_csv('dataset\경쟁사이용.csv', encoding='utf-8')
prodcat = pd.read_csv('dataset\상품분류.csv', encoding='utf-8')
purprd1 = pd.read_csv('dataset\purprd.txt', encoding='cp949')
purprd = pd.read_csv('dataset\purprd_new2.csv', encoding='utf-8', index_col=0)
membership = pd.read_csv('dataset\멤버십여부.csv', encoding='utf-8')
cust = pd.read_csv('dataset\고객DEMO.csv', encoding='utf-8')

C:\Anaconda3\envs\cakd3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## 제외 데이터
1. channel: 채널이용 데이터
2. compuse: 경쟁사 이용현황 데이터

In [132]:
# 1. channel: 채널이용 데이터
# 채널 이용의 기준일시를 알 수 없으므로 모델에 적용하기 어려움
channel.head(3)

,고객번호,제휴사,이용횟수
0,7,A_MOBILE/APP,4
1,14,A_MOBILE/APP,1
2,42,B_MOBILE/APP,23


In [133]:
# 2. compuse: 경쟁사 이용현황 데이터
# 데이터의 이용년월의 최소값이 2015년 1월이므로 학습 데이터에 사용할 수 없음
print(compuse['이용년월'].min())
compuse.head(3)

201501


,고객번호,제휴사,경쟁사,이용년월
0,2,D,D02,201507
1,51,D,D01,201504
2,77,D,D02,201503


## 사용 데이터
1. prodcat: 제휴사별 판매물품의 대/중/소분류 및 중/소분류명 정보 데이터
2. purprd: 고객별 영수증 데이터
3. membership: 고객별 멤버십 가입 데이터
4. cust: 고객별 인구 통계학적 정보 데이터


In [134]:
# 1. prodcat: 제휴사별 판매물품의 대/중/소분류 및 중/소분류명 정보 데이터
prodcat.head(3)

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명
0,A,1,101,A010101,일용잡화,위생세제
1,A,1,101,A010102,일용잡화,휴지류
2,A,1,101,A010103,일용잡화,뷰티상품


In [135]:
# 제휴사별 대/중/소 분류 체계가 다름
# 대분류코드 일원화 실시. 대분류코드보다 상위 개념인 대대분류코드도 새로 만들어 purprd에 적용함

for i in ['A','B','C','D']:
    counts = prodcat[(prodcat['제휴사']==i)]['대분류코드'].nunique(),\
            prodcat[(prodcat['제휴사']==i)]['중분류코드'].nunique(),\
            prodcat[(prodcat['제휴사']==i)]['소분류코드'].nunique()
    print(f'제휴사 {i}의 대/중/소 분류코드 수:', counts)

제휴사 A의 대/중/소 분류코드 수: (9, 50, 630)
제휴사 B의 대/중/소 분류코드 수: (92, 626, 2624)
제휴사 C의 대/중/소 분류코드 수: (17, 98, 987)
제휴사 D의 대/중/소 분류코드 수: (8, 33, 145)


In [136]:
# 2. purprd: 고객별 영수증 데이터
# prodcat의 카테고리 분류체계 변경을 반영함
# 대분류코드, 중분류코드, 소분류코드 -> 대대분류코드, 새대분류코드, 중분류코드, 소분류코드
display(purprd1.head(3), purprd.head(3))

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,15,1504,B150401,17218,44,20140222,20,2420
1,B,8664000,16,1601,B160101,17218,44,20140222,20,1070
2,B,8664000,16,1602,B160201,17218,44,20140222,20,8060


,제휴사,영수증번호,대대분류코드,새대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,2,15,1504,B150401,17218,44,20140222,20,2420
1,B,8664007,2,15,1504,B150401,11303,44,20140222,13,2400
2,B,8919520,2,15,1504,B150401,13210,48,20150921,19,5250


In [137]:
for i in ['A','B','C','D']:
    counts = purprd[(purprd['제휴사']==i)]['대대분류코드'].nunique(),\
            purprd[(purprd['제휴사']==i)]['새대분류코드'].nunique()
    print(f'제휴사 {i}의 대대/새대 분류코드 수:', counts)
pd.pivot_table(data=purprd, columns='제휴사', index='대대분류코드', aggfunc='size')

제휴사 A의 대대/새대 분류코드 수: (12, 32)
제휴사 B의 대대/새대 분류코드 수: (11, 32)
제휴사 C의 대대/새대 분류코드 수: (10, 31)
제휴사 D의 대대/새대 분류코드 수: (3, 6)


제휴사,A,B,C,D
대대분류코드,,,,
1,964059.0,3646378.0,3049651.0,NaN
2,1900369.0,6780594.0,5305150.0,NaN
3,329054.0,1362970.0,787459.0,66979.0
4,337466.0,485911.0,137762.0,37251.0
5,176462.0,188155.0,15435.0,1172.0
6,36409.0,245254.0,37039.0,NaN
7,3835.0,89381.0,10519.0,NaN
8,57119.0,3445.0,6603.0,NaN
9,1074096.0,278128.0,18062.0,NaN


In [138]:
# 3. membership: 고객별 멤버십 가입 데이터
membership.head(3)

,고객번호,멤버십명,가입년월
0,11,하이마트,201512
1,21,하이마트,201506
2,37,하이마트,201306


In [139]:
# 4. cust: 고객별 인구 통계학적 정보 데이터
cust.head(3)

,고객번호,성별,연령대,거주지역
0,1,M,60세이상,60.0
1,2,M,60세이상,100.0
2,3,M,60세이상,33.0


# 반기별 고객구매 df 만들기
- cust 데이터 기반, 2014년 1/2반기, 2015년 1/2반기의 고객별 구매정보 데이터 프레임 형성

In [140]:
# purprd 수정

# 1. 구매일자 datetime 형식으로 변환
purprd.구매일자 = pd.to_datetime(purprd.구매일자, format = '%Y%m%d')

# 2. 구매시간 범주화
def time_cat(x):
    if 9 <= x < 12: return '오전'
    elif 12 <= x < 15: return '낮'
    elif 15 <= x < 18: return '오후'
    elif 18 <= x < 21: return '저녁'
    else: return '야간'    
purprd.구매시간 = purprd.구매시간.apply(time_cat)

# 3. 점포코드를 제휴사와 합친 형태로 변환 (ex. 제휴사 B, 점포코드 101 -> 점포코드 B101)
purprd.점포코드 = purprd.제휴사 + purprd.점포코드.astype(str)

# 4. purprd 반기별 분할
purprd_14_half1 = purprd[(purprd.구매일자>=pd.to_datetime(20140101, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20140701, format='%Y%m%d'))]
purprd_14_half2 = purprd[(purprd.구매일자>=pd.to_datetime(20140701, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20150101, format='%Y%m%d'))]
purprd_15_half1 = purprd[(purprd.구매일자>=pd.to_datetime(20150101, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20150701, format='%Y%m%d'))]
purprd_15_half2 = purprd[(purprd.구매일자>=pd.to_datetime(20150701, format='%Y%m%d'))&(purprd.구매일자<pd.to_datetime(20160101, format='%Y%m%d'))]
purprd_14_half1.head(3)

,제휴사,영수증번호,대대분류코드,새대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,2,15,1504,B150401,17218,B44,2014-02-22,저녁,2420
1,B,8664007,2,15,1504,B150401,11303,B44,2014-02-22,낮,2400
21,B,9427551,2,15,1504,B150401,14795,B71,2014-04-08,저녁,6400


In [141]:
# cust에 membership 가입년월 컬럼 추가
# 여러가지 멤버십 복수가입 시, 가장 일찍 가입한 멤버십 기준으로 가입년월 집계함(min)
# 가입여부가 아닌 가입년월인 이유: 반기별로 고객데이터 분할한 이후에 가입여부가 바뀔 수 있기 때문
# 반기별 고객데이터 분할 이후 가입년월을 가입여부로 변경

membership.가입년월 = pd.to_datetime(membership.가입년월, format = '%Y%m')
cust = cust.merge(membership.pivot_table(index = '고객번호', values = '가입년월', aggfunc='min'), how = 'left', on = '고객번호').rename({'가입년월':'멤버십가입년월'}, axis = 1)
cust.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입년월
0,1,M,60세이상,60.0,NaT
1,2,M,60세이상,100.0,NaT
2,3,M,60세이상,33.0,NaT


In [142]:
# 반기별 고객구매 df 만들기: cust를 반기별로 복사

cust_14_half1 = cust.copy()
cust_14_half2 = cust.copy()
cust_15_half1 = cust.copy()
cust_15_half2 = cust.copy()

In [143]:
# 변수수정: 멤버십가입년월 -> 멤버십가입여부

ref_date = pd.to_datetime(20140701, format = '%Y%m%d')
cust_14_half1.멤버십가입년월 = cust_14_half1.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)
ref_date = pd.to_datetime(20150101, format = '%Y%m%d')
cust_14_half2.멤버십가입년월 = cust_14_half2.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)
ref_date = pd.to_datetime(20150701, format = '%Y%m%d')
cust_15_half1.멤버십가입년월 = cust_15_half1.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)
ref_date = pd.to_datetime(20160101, format = '%Y%m%d')
cust_15_half2.멤버십가입년월 = cust_15_half2.멤버십가입년월.map(lambda x: 1 if x < ref_date else 0)

cust_14_half1 = cust_14_half1.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_14_half2 = cust_14_half2.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_15_half1 = cust_15_half1.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_15_half2 = cust_15_half2.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부
0,1,M,60세이상,60.0,0
1,2,M,60세이상,100.0,0
2,3,M,60세이상,33.0,0


# 파생변수 생성

## 1. 제휴사별 구매금액비율

In [144]:
def get_amount_rate(purprd, cust):
    temp = purprd.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0)
    temp['총계'] = temp.sum(axis = 1)
    temp = temp.apply(lambda x: x/temp['총계'])
    temp = temp.drop('총계', axis = 1)
    cust = cust.merge(temp, how = 'left', on = '고객번호').rename({'A':'A구매금액비율', 'B':'B구매금액비율', 'C':'C구매금액비율', 'D':'D구매금액비율'}, axis = 1)
    return cust

cust_14_half1 = get_amount_rate(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_amount_rate(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_amount_rate(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_amount_rate(purprd_15_half2,cust_15_half2)

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0


## 2. 주요구매시간대

In [145]:
import numpy as np
from scipy.stats import mode

def get_feq_time(purprd, cust):
    temp = purprd.groupby(['고객번호', '영수증번호']).구매시간.first().reset_index().groupby('고객번호').구매시간.apply(lambda x: mode(x)[0][0])
    cust = cust.merge(temp.rename('주요구매시간대'), how = 'left', on = '고객번호')
    return cust

cust_14_half1 = get_feq_time(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_feq_time(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_feq_time(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_feq_time(purprd_15_half2,cust_15_half2)

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮


## 3. 제휴사별 방문횟수


In [146]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,A방문횟수,B방문횟수,C방문횟수,D방문횟수
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,154.0,1.0,4.0,0.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,187.0,1.0,0.0,0.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,1.0,0.0,127.0,0.0


## 4. 제휴사별 상품구매개수

In [147]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,A방문횟수,B방문횟수,C방문횟수,D방문횟수,A상품구매개수,B상품구매개수,C상품구매개수,D상품구매개수
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,154.0,1.0,4.0,0.0,230.0,1.0,7.0,0.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,187.0,1.0,0.0,0.0,338.0,1.0,0.0,0.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,1.0,0.0,127.0,0.0,1.0,0.0,347.0,0.0


## 5. 총구매금액


In [148]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')

# # 2014-1반기 기준, 각 반기의 매출 증감에 대해 보정
# r2 = cust_14_half1.총구매금액.sum() / cust_14_half2.총구매금액.sum()
# r3 = cust_14_half1.총구매금액.sum() / cust_15_half1.총구매금액.sum()
# r4 = cust_14_half1.총구매금액.sum() / cust_15_half2.총구매금액.sum()

# cust_14_half2.총구매금액 = cust_14_half2.총구매금액*r2
# cust_15_half1.총구매금액 = cust_15_half1.총구매금액*r3
# cust_15_half2.총구매금액 = cust_15_half2.총구매금액*r4

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,A방문횟수,B방문횟수,C방문횟수,D방문횟수,A상품구매개수,B상품구매개수,C상품구매개수,D상품구매개수,총구매금액
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,154.0,1.0,4.0,0.0,230.0,1.0,7.0,0.0,19925355.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,187.0,1.0,0.0,0.0,338.0,1.0,0.0,0.0,24323230.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,1.0,0.0,127.0,0.0,1.0,0.0,347.0,0.0,851215.0


## 6. 방문점포종류수

In [149]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,A방문횟수,B방문횟수,C방문횟수,D방문횟수,A상품구매개수,B상품구매개수,C상품구매개수,D상품구매개수,총구매금액,방문점포종류수
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,154.0,1.0,4.0,0.0,230.0,1.0,7.0,0.0,19925355.0,6.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,187.0,1.0,0.0,0.0,338.0,1.0,0.0,0.0,24323230.0,3.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,1.0,0.0,127.0,0.0,1.0,0.0,347.0,0.0,851215.0,2.0


## 7. 마지막구매일자

In [150]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.마지막구매일자 = cust_14_half1.마지막구매일자.astype('datetime64')
cust_14_half2.마지막구매일자 = cust_14_half2.마지막구매일자.astype('datetime64')
cust_15_half1.마지막구매일자 = cust_15_half1.마지막구매일자.astype('datetime64')
cust_15_half2.마지막구매일자 = cust_15_half2.마지막구매일자.astype('datetime64')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,...,B방문횟수,C방문횟수,D방문횟수,A상품구매개수,B상품구매개수,C상품구매개수,D상품구매개수,총구매금액,방문점포종류수,마지막구매일자
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,...,1.0,4.0,0.0,230.0,1.0,7.0,0.0,19925355.0,6.0,2014-06-30
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,...,1.0,0.0,0.0,338.0,1.0,0.0,0.0,24323230.0,3.0,2014-06-30
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,...,0.0,127.0,0.0,1.0,0.0,347.0,0.0,851215.0,2.0,2014-06-30


## 8. 회당구매금액

In [151]:
cust_14_half1['회당구매금액'] = cust_14_half1.총구매금액/(cust_14_half1.A방문횟수 + cust_14_half1.B방문횟수 + cust_14_half1.C방문횟수 + cust_14_half1.D방문횟수)
cust_14_half2['회당구매금액'] = cust_14_half2.총구매금액/(cust_14_half2.A방문횟수 + cust_14_half2.B방문횟수 + cust_14_half2.C방문횟수 + cust_14_half2.D방문횟수)
cust_15_half1['회당구매금액'] = cust_15_half1.총구매금액/(cust_15_half1.A방문횟수 + cust_15_half1.B방문횟수 + cust_15_half1.C방문횟수 + cust_15_half1.D방문횟수)
cust_15_half2['회당구매금액'] = cust_15_half2.총구매금액/(cust_15_half2.A방문횟수 + cust_15_half2.B방문횟수 + cust_15_half2.C방문횟수 + cust_15_half2.D방문횟수)

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,...,C방문횟수,D방문횟수,A상품구매개수,B상품구매개수,C상품구매개수,D상품구매개수,총구매금액,방문점포종류수,마지막구매일자,회당구매금액
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,...,4.0,0.0,230.0,1.0,7.0,0.0,19925355.0,6.0,2014-06-30,125316.698113
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,...,0.0,0.0,338.0,1.0,0.0,0.0,24323230.0,3.0,2014-06-30,129378.882979
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,...,127.0,0.0,1.0,0.0,347.0,0.0,851215.0,2.0,2014-06-30,6650.117188


## 9. 구매상품종류수
- 소분류 기준

In [152]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,...,D방문횟수,A상품구매개수,B상품구매개수,C상품구매개수,D상품구매개수,총구매금액,방문점포종류수,마지막구매일자,회당구매금액,구매상품종류수
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,...,0.0,230.0,1.0,7.0,0.0,19925355.0,6.0,2014-06-30,125316.698113,71.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,...,0.0,338.0,1.0,0.0,0.0,24323230.0,3.0,2014-06-30,129378.882979,78.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,...,0.0,1.0,0.0,347.0,0.0,851215.0,2.0,2014-06-30,6650.117188,74.0


## 10. 제휴사별 구매금액

In [153]:
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,...,D상품구매개수,총구매금액,방문점포종류수,마지막구매일자,회당구매금액,구매상품종류수,A구매금액,B구매금액,C구매금액,D구매금액
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,...,0.0,19925355.0,6.0,2014-06-30,125316.698113,71.0,19703210.0,21800.0,200345.0,0.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,...,0.0,24323230.0,3.0,2014-06-30,129378.882979,78.0,24292730.0,30500.0,0.0,0.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,...,0.0,851215.0,2.0,2014-06-30,6650.117188,74.0,80000.0,0.0,771215.0,0.0


## 11. 대대분류별 구매순위 등급
- 4등급으로 분류

In [154]:
# 분기별 구매데이터프레임 분할(총 8분기)
# fh: first half(상반기), sh: second half(하반기)

fh14_1 = purprd[purprd.구매일자<'20140401']
fh14_2 = purprd[(purprd.구매일자>='20140401')&(purprd.구매일자<'20140701')]
sh14_1 = purprd[(purprd.구매일자>='20140701')&(purprd.구매일자<'20141001')]
sh14_2 = purprd[(purprd.구매일자>='20141001')&(purprd.구매일자<'20150101')]

fh15_1 = purprd[(purprd.구매일자>='20150101')&(purprd.구매일자<'20150401')]
fh15_2 = purprd[(purprd.구매일자>='20150401')&(purprd.구매일자<'20150701')]
sh15_1 = purprd[(purprd.구매일자>='20150701')&(purprd.구매일자<'20151001')]
sh15_2 = purprd[(purprd.구매일자>='20151001')&(purprd.구매일자<'20160101')]


# 고객별 대대분류별 구매금액 총액에 대해 순위 매기기(오름차순)
# 카테고리 구매가 없는 고객은 모두 1위로 지정

fh14_1_pv = fh14_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
fh14_2_pv = fh14_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh14_1_pv = sh14_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh14_2_pv = sh14_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)

fh15_1_pv = fh15_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
fh15_2_pv = fh15_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh15_1_pv = sh15_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)
sh15_2_pv = sh15_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().rank(method='min', ascending=True).fillna(1)

# 순위에 따라 4등급으로 분류하는 함수 작성(총 고객수의 4분위수 기준으로 분류)
def rank(x):
    if x <= 4846: return 1
    elif x <= 9692: return 2
    elif x <= 14539: return 3
    else: return 4

# 전체 8분기, 각 대분류 12개에 대한 등급 매기기
for el in [fh14_1_pv, fh14_2_pv, sh14_1_pv, sh14_2_pv, fh15_1_pv, fh15_2_pv, sh15_1_pv, sh15_2_pv]:
    for i in range(1,13):
        el[i] = el[i].apply(lambda x:rank(x))

# 반기별 데이터프레임에 합병
cust_14_half1 = cust_14_half1.merge(fh14_1_pv, on='고객번호')
cust_14_half1 = cust_14_half1.merge(fh14_2_pv, on='고객번호', suffixes=['_1','_2'])
cust_14_half2 = cust_14_half2.merge(sh14_1_pv, on='고객번호')
cust_14_half2 = cust_14_half2.merge(sh14_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_15_half1 = cust_15_half1.merge(fh15_1_pv, on='고객번호')
cust_15_half1 = cust_15_half1.merge(fh15_2_pv, on='고객번호', suffixes=['_1','_2'])
cust_15_half2 = cust_15_half2.merge(sh15_1_pv, on='고객번호')
cust_15_half2 = cust_15_half2.merge(sh15_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,...,3_2,4_2,5_2,6_2,7_2,8_2,9_2,10_2,11_2,12_2
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,...,1,4,1,1,1,1,4,3,2,1
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,...,3,4,2,1,1,1,3,3,1,1
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,...,1,1,1,1,1,1,1,1,1,1


## 12. 대대분류별 구매금액

In [155]:
cust_category_14_half1 = purprd_14_half1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().fillna(0)
cust_category_14_half2 = purprd_14_half2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().fillna(0)
cust_category_15_half1 = purprd_15_half1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().fillna(0)
cust_category_15_half2 = purprd_15_half2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack().fillna(0)

cust_14_half1 = cust_14_half1.merge(cust_category_14_half1, on='고객번호', how='left')
cust_14_half2 = cust_14_half2.merge(cust_category_14_half2, on='고객번호', how='left')
cust_15_half1 = cust_15_half1.merge(cust_category_15_half1, on='고객번호', how='left')
cust_15_half2 = cust_15_half2.merge(cust_category_15_half2, on='고객번호', how='left')

cust_14_half1.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입여부,A구매금액비율,B구매금액비율,C구매금액비율,D구매금액비율,주요구매시간대,...,3,4,5,6,7,8,9,10,11,12
0,1,M,60세이상,60.0,0,0.988851,0.001094,0.010055,0.0,낮,...,677770.0,1709150.0,177460.0,0.0,0.0,0.0,10809560.0,1782410.0,175250.0,0.0
1,2,M,60세이상,100.0,0,0.998746,0.001254,0.000000,0.0,오후,...,1220600.0,7972020.0,466380.0,30500.0,0.0,1421700.0,6509990.0,1348010.0,421100.0,0.0
2,3,M,60세이상,33.0,0,0.093983,0.000000,0.906017,0.0,낮,...,6080.0,16870.0,0.0,0.0,0.0,0.0,80000.0,0.0,0.0,0.0


In [156]:
# 고객별, 분기별 구매 데이터 저장

cust_14_half1.to_csv('dataset/cust_14_half1.csv', encoding='utf-8', index=False)
cust_14_half2.to_csv('dataset/cust_14_half2.csv', encoding='utf-8', index=False)
cust_15_half1.to_csv('dataset/cust_15_half1.csv', encoding='utf-8', index=False)
cust_15_half2.to_csv('dataset/cust_15_half2.csv', encoding='utf-8', index=False)